# 1. Setup
Install Confluent-Kafka client

In [ ]:
! pip install confluent-kafka

You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-dada97a2-59f3-4e34-a482-bc1436edd614/bin/python -m pip install --upgrade pip' command.


In [ ]:
! pip show confluent-kafka

Name: confluent-kafka
Version: 2.3.0
Summary: Confluent's Python client for Apache Kafka
Home-page: https://github.com/confluentinc/confluent-kafka-python
Author: Confluent Inc
Author-email: support@confluent.io
License: 
Location: /local_disk0/.ephemeral_nfs/envs/pythonEnv-dada97a2-59f3-4e34-a482-bc1436edd614/lib/python3.9/site-packages
Requires: 
Required-by: 


# 2. Batch Processing

## 2.1. Load topic data from Confluent in batch mode

In [ ]:
from pyspark.sql import functions
topic_name = "topic_1"
bootstrap_servers = "pkc-4rn2p.canadacentral.azure.confluent.cloud:9092"
df_kafka = spark.read.format("kafka")\
    .option("kafka.bootstrap.servers", bootstrap_servers)\
    .option("subscribe", topic_name)\
    .option("kafka.security.protocol","SASL_SSL")\
    .option("kafka.sasl.mechanism", "PLAIN")\
    .option("kafka.sasl.jaas.config", "kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username=\"DMEFD5LIKVBOJCWD\" password=\"mUEgMQvRZHnL4ZqtkTUww6Dywxxr9HoDmmgzYW6jRGumni3FrstD3c0ERxplHc8j\";")\
    .load()

display(df_kafka)

key,value,topic,partition,offset,timestamp,timestampType
Mw==,eyJuYW1lIjogIkpvaG4gRG9lIiwgImFnZSI6IDM1LCAiZW1haWwiOiAiam9obi5kb2VAZ21haWwuY29tIn0=,topic_1,5,0,2024-03-30T21:30:30.634+0000,0
MQ==,eyJuYW1lIjogIkphbmUgRG9lIiwgImFnZSI6IDI5LCAiZW1haWwiOiAiamFuZS5kb2VAZ21haWwuY29tIn0=,topic_1,3,0,2024-03-30T21:30:30.604+0000,0
Mg==,eyJuYW1lIjogIkZhdGloIiwgImFnZSI6IDI0LCAiZW1haWwiOiAiZi5uYXllYmlAZ21haWwuY29tIn0=,topic_1,2,0,2024-03-30T21:30:30.736+0000,0


## 2.2. Write a Kafka sink for batch queries

### 2.2.1. Create sample data

In [ ]:
# import pyspark class Row from module sql
from pyspark.sql import *

In [ ]:
# Create the sample data
sample_data = Row("key", "value", "topic")
sample1 = sample_data('1', '{"name": "Jane Doe", "age": 29, "email": "jane.doe@gmail.com"}', "test-topic")
sample2 = sample_data('2', '{"name": "Fatih", "age": 24, "email": "f.nayebi@gmail.com"}', "test-topic")
sample3 = sample_data('3', '{"name": "John Doe", "age": 35, "email": "john.doe@gmail.com"}', "test-topic")

print(sample1)

Row(key='1', value='{"name": "Jane Doe", "age": 29, "email": "jane.doe@gmail.com"}', topic='test-topic')


### 2.2.2. Create a dataframe from sample data

In [ ]:
df = spark.createDataFrame([sample1, sample2, sample3])

In [ ]:
df.show()

+---+--------------------+----------+
|key|               value|     topic|
+---+--------------------+----------+
|  1|{"name": "Jane Do...|test-topic|
|  2|{"name": "Fatih",...|test-topic|
|  3|{"name": "John Do...|test-topic|
+---+--------------------+----------+



In [ ]:
display(df)

key,value,topic
1,"{""name"": ""Jane Doe"", ""age"": 29, ""email"": ""jane.doe@gmail.com""}",test-topic
2,"{""name"": ""Fatih"", ""age"": 24, ""email"": ""f.nayebi@gmail.com""}",test-topic
3,"{""name"": ""John Doe"", ""age"": 35, ""email"": ""john.doe@gmail.com""}",test-topic


### 2.2.3. Write data from a dataframe to a confluent kafka topic

In [ ]:
# Write key-value data from a DataFrame to a specific Kafka topic specified in an option
ds = df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")\
    .write.format("kafka")\
    .option("kafka.bootstrap.servers", bootstrap_servers)\
    .option("subscribe", topic_name)\
    .option("kafka.security.protocol","SASL_SSL")\
    .option("kafka.sasl.mechanism", "PLAIN")\
    .option("kafka.sasl.jaas.config", "kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username=\"DMEFD5LIKVBOJCWD\" password=\"mUEgMQvRZHnL4ZqtkTUww6Dywxxr9HoDmmgzYW6jRGumni3FrstD3c0ERxplHc8j\";")\
    .option("topic", topic_name)\
    .save()

# 3. Stream Processing

## 3.1. Read a stream from Kafka

In [ ]:
df_stream = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", bootstrap_servers)\
    .option("subscribe", topic_name)\
    .option("kafka.security.protocol","SASL_SSL")\
    .option("kafka.sasl.mechanism", "PLAIN")\
    .option("kafka.sasl.jaas.config", "kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username=\"DMEFD5LIKVBOJCWD\" password=\"mUEgMQvRZHnL4ZqtkTUww6Dywxxr9HoDmmgzYW6jRGumni3FrstD3c0ERxplHc8j\";")\
    .load() \

display(df_stream)

## 3.2. Write a Kafka sink for streaming queries

In [ ]:
# Write key-value data from a DataFrame to a specific Kafka topic specified in an option
ds = df_stream \
  .selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)") \
  .writeStream \
  .format("kafka") \
    .option("kafka.bootstrap.servers", bootstrap_servers)\
    .option("subscribe", topic_name)\
    .option("kafka.security.protocol","SASL_SSL")\
    .option("kafka.sasl.mechanism", "PLAIN")\
    .option("kafka.sasl.jaas.config", "kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username=\"DMEFD5LIKVBOJCWD\" password=\"mUEgMQvRZHnL4ZqtkTUww6Dywxxr9HoDmmgzYW6jRGumni3FrstD3c0ERxplHc8j\";")\
  .option("topic", "databricks_test") \
  .option("checkpointLocation", "/dbfs/dir") \
  .start()